In [ ]:
import csv, json
from osgeo import ogr

In [ ]:
# Transform NI Small Areas shapefile into GeoJSON
cmd = "ogr2ogr -f GeoJSON -t_srs crs:84 SA2011/SA2011.geojson SA2011/SA2011.shp"
!$cmd

In [ ]:
# Load small areas GeoJSON file
with open("SA2011/SA2011.geojson") as f:
    sa2011 = json.load(f)

In [ ]:
#sa2011['features'][0]['properties']
#{u'Hectares': 169.044,
# u'SA2011': u'N00000002',
# u'SOA2011': u'95AA01S2',
# u'X_COORD': 315465.0,
# u'Y_COORD': 377176.0}

In [ ]:
belfastData = []
with open("Geographic Data (statistical geographies).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['HSCT'] == "Belfast HSCT":
            belfastData.append(row)

In [ ]:
#belfastData[0]
#{'AA1998': 'BELFAST WEST',
# 'AA2008': 'BELFAST WEST',
# 'HSCT': 'Belfast HSCT',
# 'LGD1992NAME': 'BELFAST',
# 'SA': 'N00000825',
# 'SOA': '95GG01S1',
# 'SQ Km': '0.11',
# 'Settlement2015(1)': 'BELFAST CITY',
# 'Settlement2015(2)': '',
# 'Settlement2015(3)': '',
# 'URBAN_RURAL(2015)': 'Urban',
# 'WARD1992': 'ANDERSONSTOWN'}

In [ ]:
belfastGeoJSON = { 
    "type": "FeatureCollection",
    "features": []
}
for row in belfastData:
    sa = row['SA']
    for feature in sa2011['features']:
        if feature['properties']['SA2011'] == sa:
            for key in row.keys():
                feature['properties'][key] = row[key]
            belfastGeoJSON['features'].append(feature)
            break
with open('belfast.geojson', 'w') as outfile:  
    json.dump(belfastGeoJSON, outfile, indent=4)